In [37]:
import numpy as np
from datetime import datetime
from kadlu.geospatial.data_sources import chs
from kadlu.geospatial.data_sources import era5
from kadlu.geospatial.data_sources import hycom
from kadlu.geospatial.data_sources import wwiii
from kadlu.geospatial.data_sources.fetch_util import database_cfg
from kadlu.geospatial.data_sources.fetch_util import dt_2_epoch

#from kadlu.geospatial.data_sources import fetch_util

conn, db = database_cfg()

In [38]:
from importlib import reload
reload(wwiii)
reload(chs)

<module 'kadlu.geospatial.data_sources.chs' from '/home/matt_s/kadlu/kadlu/geospatial/data_sources/chs.py'>

In [43]:
temp_salinity_source = hycom.Hycom()
bathysource = chs.Chs()
wavesource1 = era5.Era5()
wavesource2 = wwiii.Wwiii()

In [44]:
# parameter selection
start = datetime(2015, 3, 9, 13, 25, 0)
end   = datetime(2015, 3, 9, 19, 0, 0)
top, bottom = 0, 0 

# gulf st lawrence
#south, west = 44.013, -70.429
#north, east = 51.823, -55.977
# no wave data for gulf st lawrence :( 

# hudsons bay
south, west = 51.388, -95.321
north, east = 64.068, -75.703
# hudsons bay subset
north, east = 55.039, -78.740

## Fetching

In [41]:
temp_salinity_source.fetch_salinity(south=south, north=north, west=west, east=east, 
                      start=start, end=end, top=top, bottom=bottom)

temp_salinity_source.fetch_temp(south=south, north=north, west=west, east=east, 
                      start=start, end=end, top=top, bottom=bottom)

downloading 57408 salinity values from hycom...
downloaded 56.2Kb in 1.364s. parsed and inserted 0 rows in 3.305s
48228 null values removed, 9180 duplicate rows ignored

downloading 57408 water_temp values from hycom...
downloaded 58.5Kb in 1.178s. parsed and inserted 0 rows in 3.196s
48228 null values removed, 9180 duplicate rows ignored



In [36]:
bathysource.fetch_bathymetry(south=south, north=north, west=west, east=east)

CA2_5500N08600W.tif: downloading 22/22 from CHS NONNA-100...
CA2_5100N07900W.tif processed and inserted 0 rows. 1001960 null values removed, 41 duplicate rows ignored
CA2_5100N08000W.tif processed and inserted 0 rows. 1001840 null values removed, 161 duplicate rows ignored
CA2_5100N08100W.tif processed and inserted 0 rows. 988733 null values removed, 13268 duplicate rows ignored
CA2_5200N07900W.tif processed and inserted 0 rows. 1001987 null values removed, 14 duplicate rows ignored
CA2_5200N08000W.tif processed and inserted 0 rows. 1001602 null values removed, 399 duplicate rows ignored
CA2_5200N08100W.tif processed and inserted 0 rows. 1001707 null values removed, 294 duplicate rows ignored
CA2_5200N08200W.tif processed and inserted 0 rows. 1001953 null values removed, 48 duplicate rows ignored
CA2_5300N08000W.tif processed and inserted 0 rows. 999628 null values removed, 2373 duplicate rows ignored
CA2_5300N08100W.tif processed and inserted 0 rows. 1001609 null values removed, 392 d

In [107]:
wavesource1.fetch_windwaveswellheight(south=south, north=north, 
                                      west=west, east=east, 
                                      start=start, end=end)

2019-11-14 15:39:41,213 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


2019-11-14 15:39:42,769 INFO Request is completed
2019-11-14 15:39:42,771 INFO Downloading http://136.156.133.39/cache-compute-0012/cache/data7/adaptor.mars.internal-1573659649.4384108-16255-4-ffc6e405-d559-45ac-afd5-d199c6aa9c57.grib to /home/matt_s/kadlu/storage/ERA5_reanalysis_significant_height_of_combined_wind_waves_and_swell_2015-03-09_13h.grb2 (312.1K)
2019-11-14 15:39:43,660 INFO Download rate 351.2K/s
2019-11-14 15:39:46,720 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


ERA5_reanalysis_significant_height_of_combined_wind_waves_and_swell_2015-03-09_13h.grb2 processed and inserted 0 rows. 116500 null values removed, 143420 duplicate rows ignored


2019-11-14 15:39:47,652 INFO Request is queued
2019-11-14 15:39:48,805 INFO Request is running
2019-11-14 15:39:50,458 INFO Request is completed
2019-11-14 15:39:50,459 INFO Downloading http://136.156.133.25/cache-compute-0008/cache/data4/adaptor.mars.internal-1573760388.1020927-14347-12-e6bf212f-048c-4202-acef-e4544f962909.grib to /home/matt_s/kadlu/storage/ERA5_reanalysis_significant_height_of_combined_wind_waves_and_swell_2015-03-09_14h.grb2 (312.1K)
2019-11-14 15:39:51,313 INFO Download rate 365.6K/s
2019-11-14 15:39:54,353 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


ERA5_reanalysis_significant_height_of_combined_wind_waves_and_swell_2015-03-09_14h.grb2 processed and inserted 0 rows. 116500 null values removed, 143420 duplicate rows ignored


...

ERA5_reanalysis_significant_height_of_combined_wind_waves_and_swell_2015-03-09_17h.grb2 processed and inserted 0 rows. 116500 null values removed, 143420 duplicate rows ignored


2019-11-14 15:40:12,698 INFO Request is completed
2019-11-14 15:40:12,699 INFO Downloading http://136.156.132.198/cache-compute-0003/cache/data3/adaptor.mars.internal-1573660574.7192633-13313-3-308e7de4-ea2a-4862-bb60-bc087a6b1ad4.grib to /home/matt_s/kadlu/storage/ERA5_reanalysis_significant_height_of_combined_wind_waves_and_swell_2015-03-09_18h.grb2 (312.1K)
2019-11-14 15:40:13,592 INFO Download rate 349.6K/s
2019-11-14 15:40:16,912 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


ERA5_reanalysis_significant_height_of_combined_wind_waves_and_swell_2015-03-09_18h.grb2 processed and inserted 0 rows. 116500 null values removed, 143420 duplicate rows ignored


2019-11-14 15:40:23,152 INFO Request is completed
2019-11-14 15:40:23,153 INFO Downloading http://136.156.132.153/cache-compute-0002/cache/data8/adaptor.mars.internal-1573660580.6612635-12075-16-5152f152-4222-47a4-ad8b-b965d44d0ce4.grib to /home/matt_s/kadlu/storage/ERA5_reanalysis_significant_height_of_combined_wind_waves_and_swell_2015-03-09_19h.grb2 (312.1K)
2019-11-14 15:40:24,060 INFO Download rate 344.3K/s


ERA5_reanalysis_significant_height_of_combined_wind_waves_and_swell_2015-03-09_19h.grb2 processed and inserted 0 rows. 116500 null values removed, 143420 duplicate rows ignored


In [9]:
wavesource2.fetch_windwaveheight(south=south, north=north, 
                                 west=west, east=east, 
                                 start=start, end=end)

/home/matt_s/kadlu/kadlu/geospatial/data_sources/fetch_util.py:133: UserWarning: No regions matched for query. Defaulting to ['glo_30m'] (1 regions)
  warnings.warn(f"No regions matched for query. Defaulting to {default} ({len(default)} regions)")


downloading multi_1.glo_30m.hs.201503.grb2 from NOAA WaveWatch III...
preparing multi_1.glo_30m.hs.201503.grb2 for the database...


KeyboardInterrupt: 

## Loading (example)

In [110]:
temp, tlat, tlon, ttime, tdep = \
temp_salinity_source.load_temp(south=south, north=north, west=west, east=east, 
                               start=start, end=end, top=top, bottom=bottom)

salinity, slat, slon, stime, sdep = \
temp_salinity_source.load_temp(south=south, north=north, west=west, east=east, 
                               start=start, end=end, top=top, bottom=bottom)

In [111]:
bathy, blat, blon = bathysource.load_bathymetry(south=south, north=north, west=west, east=east)

In [114]:
wave, wlat, wlon, wtime = \
wavesource1.load_windwaveswellheight(south=south, north=north, 
                                     west=west, east=east, 
                                     start=start, end=end)

ValueError: not enough values to unpack (expected 5, got 0)

In [115]:
wave, wlat, wlon, wtime = \
wavesource2.load_windwaveheight(south=south, north=north, 
                                west=west, east=east, 
                                start=start, end=end)

ValueError: not enough values to unpack (expected 5, got 0)

## SoundSpeed -- loading data

In [45]:
from kadlu.geospatial.ocean import Ocean
from kadlu.sound.sound_speed import SoundSpeed 
from kadlu.geospatial.data_sources.fetch_util import storage_cfg

In [46]:
step = 1 # 1 deg -> approx. 100km at equator 

for ymin in np.arange(south, north, step):
    for xmin in np.arange(west, east, step):
        xmax = xmin + step
        ymax = ymin + step
        
        try:
            bathy, blat, blon = bathysource.load_bathymetry(south=ymin, north=ymax, west=xmin, east=xmax)
        except:
            print(f"no bathy data for {(ymin, xmin)}, skipping")
            continue
        
        temp, tlat, tlon, ttime, tdep = \
        temp_salinity_source.load_temp(south=ymin, north=ymax, west=xmin, east=xmax, 
                                       start=start, end=end, top=top, bottom=bottom)

        salinity, slat, slon, stime, sdep = \
        temp_salinity_source.load_temp(south=ymin, north=ymax, west=xmin, east=xmax, 
                                       start=start, end=end, top=top, bottom=bottom)
        """
        try:
            wave, wlat, wlon, wtime = \
            wavesource2.load_windwaveheight(south=ymin, north=ymax, 
                                                 west=xmin, east=xmax, 
                                                 start=start, end=end)
        except Exception as e :
            print(f"no wave data for {(ymin, xmin)}", end='\t')
            print(e)
            continue
        """
        
        wave = np.array([5 for x in temp])
        
        oceantest = Ocean(salinity=salinity, temp=temp, bathy=bathy, wave=wave, wave_var='comb. wind wave swell height')
        oceantest.set_origin(lat_ref=tlat, lon_ref=tlon)
        
        sndspd = SoundSpeed(oceantest)._sound_speed(tlat, tlon, tdep, temp, salinity)
        
        print(sndspd, ymin, xmin)


no bathy data for (51.388, -95.321), skipping
no bathy data for (51.388, -94.321), skipping
no bathy data for (51.388, -93.321), skipping
no bathy data for (51.388, -92.321), skipping
no bathy data for (51.388, -91.321), skipping
no bathy data for (51.388, -90.321), skipping
no bathy data for (51.388, -89.321), skipping
no bathy data for (51.388, -88.321), skipping
no bathy data for (51.388, -87.321), skipping
no bathy data for (51.388, -86.321), skipping
no bathy data for (51.388, -85.321), skipping
no bathy data for (51.388, -84.321), skipping
no bathy data for (51.388, -83.321), skipping


TypeError: loop of ufunc does not support argument 0 of type float which has no callable cos method

In [16]:
temp

NameError: name 'temp' is not defined

In [11]:
wavesource1.load_windwaveswellheight(south=south, north=north, west=west, east=east, 
                                       start=start, end=end)

ValueError: not enough values to unpack (expected 5, got 0)